In [2]:
import pandas as pd
import numpy as np
import requests 

# Thu thập dữ liệu về thủ đô
## Đọc dữ liệu
Dữ liệu chứa toạ độ của các vùng lãnh thổ trên thế giới

In [74]:
capital_pos = pd.read_csv("https://gist.githubusercontent.com/ofou/df09a6834a8421b4f376c875194915c9/raw/355eb56e164ddc3cd1a9467c524422cb674e71a9/country-capital-lat-long-population.csv")

Các nước thuộc thành viên của Liên Hợp Quốc

In [75]:
country_UN = pd.read_html("https://www.worlddata.info/alliances/un-united-nations.php")[0]
country_UN.shape

(193, 4)

## Nối 2 bảng

In [76]:
c_merge = capital_pos.merge(country_UN,how='outer',left_on='Country', right_on = "Country",indicator=True)
lo = c_merge['Country'][c_merge._merge == 'left_only']
ro = c_merge['Country'][c_merge._merge == 'right_only']

Tìm ra vị trí cần thay thế 

In [77]:
index_true = [24,29,33,39,60,207,200,101,53,115,136,169,59,205,172,168,203,221,222,227,228]
cap_pos_copy = capital_pos.copy()

In [78]:
def update_country(x):
    return list(ro)[index_true.index(x.name)] if x.name in index_true else x.Country
cap_pos_copy['Country'] = cap_pos_copy.apply(update_country, axis=1)

In [79]:
capital = cap_pos_copy.merge(country_UN,how='inner',left_on='Country', right_on = "Country")
capital.drop(columns=['Population_x','Capital Type'],inplace=True)

In [80]:
capital.shape

(193, 7)

Vì tên thủ đô nhiều nước bị sai nên tìm lại dữ liệu chứa tên thủ đô chính xác và tiếp tục nối

In [81]:
cap = pd.read_html("https://catking.in/list-of-countries-capitals-currencies-of-the-world/")[0].drop(0, axis = 1).drop(0, axis = 0)

In [82]:
cap.rename({1:"Country",2:"Capital",3:'Currency'}, axis = 1, inplace=True)
cap.reset_index(inplace=True)

In [83]:
c = capital.merge(cap[['Country','Capital']],how='outer',left_on='Country', right_on = 'Country', indicator=True)
l = c.Country[c._merge == 'left_only']
r = c.Country[c._merge == 'right_only']
i_true = [193,194,195,197,198,201,196,200,205,206,202,208,204,199,210]
cap = cap.replace(list(r[i_true]),list(l)).drop(['index'],axis = 1)

In [84]:
country_capital = capital.merge(cap[['Country','Capital']],how='inner',left_on='Country', right_on = 'Country').drop(['Capital City'],axis = 1)

## Xuất dữ liệu về các thủ đô và quốc gia
Các quốc gia đều là thành viên chính thức của Liên Hợp Quốc

In [85]:
country_capital

,Country,Latitude,Longitude,Commencement,Population_y,Area,Capital
0,Afghanistan,34.5289,69.1725,1946,41.13 M,"653,000 km²",Kabul
1,Albania,41.3275,19.8189,1955,2.78 M,"29,000 km²",Tirane
2,Algeria,36.7525,3.0420,1962,44.90 M,"2,382,000 km²",Algiers
3,Andorra,42.5078,1.5211,1993,0.08 M,470 km²,Andorra la Vella
4,Angola,-8.8368,13.2343,1976,35.59 M,"1,247,000 km²",Luanda
...,...,...,...,...,...,...,...
188,Venezuela,10.4880,-66.8792,1945,28.30 M,"912,000 km²",Caracas
189,Vietnam,21.0245,105.8412,1977,98.19 M,"331,000 km²",Hanoi
190,Yemen,15.3531,44.2078,1947,33.70 M,"528,000 km²",Sanaa
191,Zambia,-15.4134,28.2771,1964,20.02 M,"753,000 km²",Lusaka


    Country: Tên quốc gia thuộc Liên Hợp Quốc
    Latitude: Vĩ độ của thủ đô
    Longitude: Kinh độ của thủ đô
    Commencement: Năm gia nhập tổ chức
    Population_y: Dân số quốc gia (triệu người)
    Area: Diện tích lãnh thổ (km^2)
    Capital: Thủ đô của quốc gia đó
    
Cột Population_y và Area cần xử lý kiểu dữ liệu và xoá bỏ đơn vị

Xuất file csv

In [90]:
country_capital.to_csv(".\BTL2\country_UN.csv",index=False)

# Thu thập dữ liệu về thời tiết
## Tiến hành thu thập
Đọc dữ liệu chứa thông tin về toạ độ

In [91]:
country = pd.read_csv("./BTL2/country_UN.csv")

In [92]:
country

,Country,Latitude,Longitude,Commencement,Population_y,Area,Capital
0,Afghanistan,34.5289,69.1725,1946,41.13 M,"653,000 km²",Kabul
1,Albania,41.3275,19.8189,1955,2.78 M,"29,000 km²",Tirane
2,Algeria,36.7525,3.0420,1962,44.90 M,"2,382,000 km²",Algiers
3,Andorra,42.5078,1.5211,1993,0.08 M,470 km²,Andorra la Vella
4,Angola,-8.8368,13.2343,1976,35.59 M,"1,247,000 km²",Luanda
...,...,...,...,...,...,...,...
188,Venezuela,10.4880,-66.8792,1945,28.30 M,"912,000 km²",Caracas
189,Vietnam,21.0245,105.8412,1977,98.19 M,"331,000 km²",Hanoi
190,Yemen,15.3531,44.2078,1947,33.70 M,"528,000 km²",Sanaa
191,Zambia,-15.4134,28.2771,1964,20.02 M,"753,000 km²",Lusaka


In [14]:
def crawl(i,lat,long):
    link_API = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={long}&start_date=2019-01-01&end_date=2023-08-01&hourly=temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,rain,snowfall,weathercode,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,et0_fao_evapotranspiration,vapor_pressure_deficit,windspeed_10m,winddirection_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm,is_day&timezone=auto"
    rq = requests.get(link_API)
    dt = rq.json()
    
    # Thêm độ cao so với mực nước biển
    lst_elevation.append(dt['elevation'])
    # Dữ liệu chứa trong key 'hourly', lưu theo index
    pd.DataFrame(dt['hourly']).to_csv('./BTL2/data/'+str(i)+'.csv',index=False)

In [16]:
lst_elevation = []
for i in range(len(country)):
    try:
        crawl(i,country.at[i, 'Latitude'], country.at[i,'Longitude'])
        print(i,country.at[i, 'Country'])
    except:
        print(i)
        break

0 Afghanistan
1 Albania
2 Algeria
3 Andorra
4 Angola
5 Antigua and Barbuda
6 Argentina
7 Armenia
8 Australia
9 Austria
10 Azerbaijan
11 Bahamas
12 Bahrain
13 Bangladesh
14 Barbados
15 Belarus
16 Belgium
17 Belize
18 Benin
19 Bhutan
20 Bolivia
21 Bosnia and Herzegovina
22 Botswana
23 Brazil
24 Brunei
25 Bulgaria
26 Burkina Faso
27 Burundi
28 Cape Verde
29 Cambodia
30 Cameroon
31 Canada
32 Central Africa
33 Chad
34 Chile
35 China
36 Colombia
37 Comoros
38 Congo
39 Costa Rica
40 Ivory Coast
41 Croatia
42 Cuba
43 Cyprus
44 Czechia
45 North Korea
46 Congo (Dem. Republic)
47 Denmark
48 Djibouti
49 Dominica
50 Dominican Republic
51 Ecuador
52 Egypt
53 El Salvador
54 Equatorial Guinea
55 Eritrea
56 Estonia
57 Ethiopia
58 Fiji
59 Finland
60 France
61 Gabon
62 Gambia
63 Georgia
64 Germany
65 Ghana
66 Greece
67 Grenada
68 Guatemala
69 Guinea
70 Guinea-Bissau
71 Guyana
72 Haiti
73 Honduras
74 Hungary
75 Iceland
76 India
77 Indonesia
78 Iran
79 Iraq
80 Ireland
81 Israel
82 Italy
83 Jamaica
84 Japan

## Xử lý bảng dữ liệu
### Dữ liệu thành phố, quốc gia
Thêm cột độ cao so với mực nước biển của từng thành phố

In [93]:
country['elevation']  = lst_elevation

Sửa kiểu dữ liệu

In [94]:
country.Population_y = country.Population_y.apply(lambda x: float(x.split()[0].replace(',','')))

In [95]:
country.Area = country.Area.apply(lambda x : int(x.split()[0].replace(',','')))

In [96]:
country.head()

,Country,Latitude,Longitude,Commencement,Population_y,Area,Capital,elevation
0,Afghanistan,34.5289,69.1725,1946,41.13,653000,Kabul,1800.0
1,Albania,41.3275,19.8189,1955,2.78,29000,Tirane,110.0
2,Algeria,36.7525,3.0420,1962,44.90,2382000,Algiers,186.0
3,Andorra,42.5078,1.5211,1993,0.08,470,Andorra la Vella,1045.0
4,Angola,-8.8368,13.2343,1976,35.59,1247000,Luanda,75.0


Xuất file csv

In [97]:
country.to_csv("./BTL2/country_UN_update.csv",index=False)

### Dữ liệu thời tiết sau khi thu thập

In [61]:
summ = 0
all_df = []
for x in range(193):
    df = pd.read_csv('./BTL2/data/'+str(x)+'.csv')
    df['City'] = country.at[x,'Capital']
    summ += df.shape[0]
    all_df.append(df)
summ

7753968

In [67]:
all_df[0].head()

,time,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,rain,snowfall,weathercode,pressure_msl,...,cloudcover_mid,cloudcover_high,et0_fao_evapotranspiration,vapor_pressure_deficit,windspeed_10m,winddirection_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm,is_day,City
0,2019-01-01T00:00,2.4,50,-7.0,-1.3,0.0,0.0,0.0,2,1023.9,...,55,95,0.01,0.37,5.2,258,0.1,0.212,0,Kabul
1,2019-01-01T01:00,2.3,50,-7.0,-1.5,0.0,0.0,0.0,2,1023.7,...,84,96,0.01,0.36,5.2,258,0.1,0.212,0,Kabul
2,2019-01-01T02:00,2.1,51,-7.1,-1.5,0.0,0.0,0.0,3,1023.7,...,93,97,0.01,0.35,4.5,256,0.0,0.212,0,Kabul
3,2019-01-01T03:00,2.0,51,-7.1,-1.8,0.0,0.0,0.0,3,1023.5,...,86,95,0.01,0.35,5.1,262,-0.0,0.212,0,Kabul
4,2019-01-01T04:00,2.0,51,-7.2,-1.7,0.0,0.0,0.0,3,1022.5,...,89,95,0.01,0.35,5.1,262,-0.0,0.212,0,Kabul


Dữ liệu tổng cộng gồm 7753968 dòng, 23 cột

Xuất file csv

In [62]:
pd.concat(all_df).to_csv("data.csv",index=False)